In [18]:
import requests
import json
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd
import urllib.request

In [21]:
import requests
page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")

In [64]:
def into_celcius(temp):
    return round((int(temp) - 32) * 5/9, 2)

days = {0:'Monday', 1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
data = {'day':[], 'date':[],'low_temp_degrees':[],'high_temp_degrees':[]}
soup=bs(page.content,"html.parser")
all=soup.find("div",{"class":"locations-title ten-day-page-title"})
table=soup.find_all(class_="DetailsSummary--DetailsSummary--QpFD- DetailsSummary--fadeOnOpen--1MLf5")
c = 0
for j,items in enumerate(table):
    #for i in range(len(items.find_all("span"))-1):
          #print(c) 
          #print(int("".join(list(items.find_all("span",{"class":"DetailsSummary--highTempValue--3x6cL"})[0].text)[0:2])))  
          data["date"].append((datetime.date.today() + datetime.timedelta(days=j)).strftime('%Y-%m-%d'))
          date = datetime.date.today() + datetime.timedelta(days=j)
          data["day"].append(days[date.weekday()]) 
          data["high_temp_degrees"].append(into_celcius(int("".join(list(items.find_all("span",{"class":"DetailsSummary--highTempValue--3x6cL"})[0].text)[0:2])))) 
          data["low_temp_degrees"].append(into_celcius(int("".join(list(items.find_all("span",{"class":"DetailsSummary--lowTempValue--1DlJK"})[0].text)[0:2]))))
          c += 1
data 

{'day': ['Wednesday',
  'Thursday',
  'Friday',
  'Saturday',
  'Sunday',
  'Monday',
  'Tuesday',
  'Wednesday',
  'Thursday',
  'Friday',
  'Saturday',
  'Sunday',
  'Monday',
  'Tuesday',
  'Wednesday'],
 'date': ['2021-03-24',
  '2021-03-25',
  '2021-03-26',
  '2021-03-27',
  '2021-03-28',
  '2021-03-29',
  '2021-03-30',
  '2021-03-31',
  '2021-04-01',
  '2021-04-02',
  '2021-04-03',
  '2021-04-04',
  '2021-04-05',
  '2021-04-06',
  '2021-04-07'],
 'low_temp_degrees': [8.33,
  6.67,
  7.78,
  7.78,
  7.78,
  7.78,
  8.89,
  9.44,
  8.33,
  7.78,
  7.22,
  6.67,
  6.67,
  7.22,
  7.22],
 'high_temp_degrees': [15.0,
  13.89,
  16.67,
  18.33,
  16.11,
  15.0,
  18.89,
  20.56,
  19.44,
  17.22,
  17.22,
  16.11,
  15.0,
  15.0,
  14.44]}

In [65]:
data = pd.DataFrame(data)
data

,day,date,low_temp_degrees,high_temp_degrees
0,Wednesday,2021-03-24,8.33,15.00
1,Thursday,2021-03-25,6.67,13.89
2,Friday,2021-03-26,7.78,16.67
3,Saturday,2021-03-27,7.78,18.33
4,Sunday,2021-03-28,7.78,16.11
5,Monday,2021-03-29,7.78,15.00
6,Tuesday,2021-03-30,8.89,18.89
7,Wednesday,2021-03-31,9.44,20.56
8,Thursday,2021-04-01,8.33,19.44
9,Friday,2021-04-02,7.78,17.22
